Инструкции:
1) Создать телеграм бота через BotFather
2) Получить у него Токен и вставить его в код вместо YOUR_BOT_TOKEN
3) Установить все необходимые библиотеки с файла "requirements.txt"
4) Данный код работает с браузером MicrosoftEdge если его нет на компьютере
надо установить (Можно поменять при желании в строчке browser = webdriver.Edge())
5) Запустить код
4) Начать чат с ботом (бот будет сразу обрабатывать сообщение /start и искать ответ)
6) После сообщения "По вашему запросу ничего не найдено"
можно задавать вопрос из списка тестовых указаных VK Cloud Case. Время обработки примерно 6 секунд

In [ ]:
import time  # Import the 'time' module for time-related operations.
from selenium.webdriver import Keys  # Import 'Keys' from 'selenium.webdriver' to use keyboard keys.
import logging  # Import the 'logging' module for logging purposes.
from aiogram import Bot, Dispatcher, types, executor  # Import modules from aiogram for creating a Telegram bot.
import requests  # Import 'requests' for making HTTP requests.
from bs4 import BeautifulSoup  # Import 'BeautifulSoup' for parsing HTML content.
from selenium import webdriver  # Import 'webdriver' for web scraping with Selenium.
from selenium.webdriver.common.by import By  # Import 'By' from 'selenium.webdriver.common.by' for locating elements.

In [ ]:
logging.basicConfig(level=logging.INFO)  # Configure logging with INFO level.

In [ ]:
bot = Bot(token='YOUR_BOT_TOKEN')  # Initialize the Telegram bot using your bot token.
dp = Dispatcher(bot)  # Create a dispatcher for handling messages.
browser = webdriver.Edge()  # Initialize the Edge webdriver for web scraping.

In [ ]:
def fetch_docs_data(question):
    browser.get("https://mcs.mail.ru/docs")  # Open the Mail.ru documentation page in the browser.

    try:
        user_input1 = browser.find_element(By.CSS_SELECTOR, "button[type='button']")  # Find and click a button to search line if it is hiden.
        user_input1.click()
    except Exception as e:
        pass

    user_input = browser.find_element(By.CSS_SELECTOR, "input[name='query']")  # Find the search input element.
    user_input.send_keys(question)  # Enter the user's question into the search input.
    user_input.send_keys(Keys.RETURN)  # Press the Enter key to initiate the search.

    try:
        time.sleep(3)  # Wait for a few seconds.
        first_link = browser.find_element(By.CSS_SELECTOR, "a.PagedList_itemLink__8GurP")  # Find the first search result link.
        first_link.click()  # Click on the first search result link.
        time.sleep(3)  # Wait for a few seconds.
        data = browser.current_url  # Get the current URL of the browser.

        result3 = ''  # Initialize a variable to store the extracted text content.

        response = requests.get(data)  # Send an HTTP GET request to the current URL.
        result = BeautifulSoup(response.text, 'html.parser')  # Parse the response using BeautifulSoup.

        for element in result.select("div", class_='MarkdownContainer_reactMarkdown__C5bXL'):
            result2 = element.select('p')  # Find paragraphs within a specific div.
            for p in result2:
                result3 = result3 + p.get_text()  # Extract and concatenate text from paragraphs.
                print(p.get_text())  # Print the text content.
                result3 = result3 + '\n'  # Add a newline between paragraphs.
            break       #to not make output message too long
        return result3 + data  # Return the extracted text content and the current URL.

    except Exception as e:
        return "По вашему запросу ничего не найдено"  # Return a message for an unsuccessful search.

In [ ]:
@dp.message_handler()
async def echo(message: types.Message):
    user_message = message.text  # Get the user's message.
    documentation_data = fetch_docs_data(user_message)  # Call the function to fetch documentation data.
    if documentation_data:
        await message.answer(documentation_data)  # Send the extracted data as a response.
    else:
        await message.answer("Повторите попытку позже")  # Send a message if the search was unsuccessful.

In [ ]:
if __name__ == "__main__":
    executor.start_polling(dp, skip_updates=True)  # Start the bot to listen for messages and execute handlers.